In [ ]:
#Failed to build scikit-image lap
#!pip install -r sort_requirements.txt

In [ ]:
#!pip install -r utils_requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import numpy as np
import pandas as pd
import torch
import cv2
import matplotlib.pyplot as plt
import os

from utils import write_metrics_to_df, write_time_to_df, replace_cuda_with_cpu, make_all_numpy,\
 video_annot_txt_to_dataframe, motMetricsEnhancedCalculator, convert_array, usingPIL

In [ ]:
#path to sequeuence on local : D:\Drone_object_tracking\VisDrone2019-MOT-val\sequences\uav0000086_00000_v
#path to sequeuence on colab /content/drive/MyDrive/VisDrone2019-MOT-val/sequences/uav0000086_00000_v
from os import walk
#uav0000086_00000_v
#uav0000268_05773_v
video_name = 'uav0000086_00000_v'
dir_path_source_sequences = 'D:\\Drone_object_tracking\\VisDrone2019-MOT-val\\sequences'
dir_path_source_annotations = 'D:\\Drone_object_tracking\\VisDrone2019-MOT-val\\annotations'
dir_path = 'D:\\Drone_object_tracking\\VisDrone2019-MOT-val\\sequences'+video_name
annot_path = 'D:\\Drone_object_tracking\\VisDrone2019-MOT-val\\annotations'+video_name+'.txt'


#filenames = next(walk(dir_path), (None, None, []))[2]  # [] if no file

In [ ]:
#%matplotlib inline
#import matplotlib.pyplot as plt
import datetime
import cv2
from time import time
def make_predictions(model=None, tracker=None, dir_path_sequences=None, img_size=None):
    '''
    input:
    model - model for predictions. MUST return predictions in formst like YOLOv8
    tracker - MOT technic. SORT or DeepSORT
    dir_path_sequences - string path to video sequence('D:\\Drone_object_tracking\\VisDrone2019-MOT-val\\sequences\\uav0000268_05773_v') as example
    img_size - image size for YOLO inference

    return:
    preds - dataframe of predictions in MOT16 format
    str_res - string with description of time characteristics of alghorithm
    times - dataframe of time wasted for different parts of function
    '''
    #print(next(walk(dir_path_sequences)))
    #print(dir_path_sequences+'/'+next(walk(dir_path_sequences)))
    height, width = cv2.imread(dir_path_sequences+'/'+next(walk(dir_path_sequences))[2][0]).shape[:2]

    fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')

    try:
        os.remove("video.avi")
    except: pass

    video = cv2.VideoWriter(f'video{os.path.basename(dir_path_sequences)}.avi', fourcc, 20, (width, height))

    mot_tracker = tracker

    #acc = mm.MOTAccumulator(auto_id=True)
    #annots = video_annot_txt_to_dataframe('D:\\Drone_object_tracking\\VisDrone2019-MOT-val\\annotations\\uav0000086_00000_v.txt')
    columns=['frame_number','obj_id','left_top_x','left_top_y','width','height','confidence', 'category','truncation',
                'occlusion']

    preds = pd.DataFrame(columns=columns)
    #print(len(filenames))

    avg_yolo_time = 0.
    avg_predictor_time = 0.
    avg_frame_time_arr= []
    avg_yolo_time_arr= []
    avg_predictor_time = []
    avg_resizing_time = []
    avg_drawing_time = []
    avg_reading_time = []

    filenames = sorted(next(walk(dir_path_sequences), (None, None, []))[2])#BUG may appear
    #print(filenames)
    #i = 0
    for idx, img_relative_path in enumerate(filenames):

        time_before_fps  = time()
        path = dir_path_sequences+'/'+img_relative_path

        time_before_reading = time()
        #img = cv2.imread(path)    #Using this, time after - before is NEARLY equal for 928px, for 640 still different(at the begging of small inference time is 20 ms)
        img = usingPIL(path)
        time_after_reading = time()
        avg_reading_time.append((time_after_reading-time_before_reading)*1000)

        time_before_resizing = time()

        time_after_resizing = time()
        avg_resizing_time.append((time_after_resizing-time_before_resizing)*1000)

        time_before_YOLO = time()
        prediction = model(img, verbose=False,imgsz=img_size)
        time_after_YOLO = time()
        boxes =  prediction[0].boxes.xyxy.type(torch.IntTensor).to('cuda')
        scores = prediction[0].boxes.conf
        classes = prediction[0].boxes.cls.type(torch.IntTensor)
        scores = torch.unsqueeze(scores, 1)

        torch_detections = torch.cat((boxes,scores), dim=1)
        torch_detections_cpu = replace_cuda_with_cpu(torch_detections)
        numpy_detections = make_all_numpy(torch_detections_cpu)

        time_before_updating_tracker = time()
        if type(mot_tracker) == DeepSort:
            numpy_detections = np.concatenate((numpy_detections,np.expand_dims(classes.cpu().numpy(),axis=1) ),
                                            axis=1)
          #print('a')
            result_tracker = mot_tracker.update_tracks(convert_array(numpy_detections), frame=img) # WARNING here
          #print('b')
            results = []

            for res in result_tracker:
                arr = res.to_tlwh().tolist()
                arr.append(res.track_id)
                results.append(arr)



        else:
            result_tracker = mot_tracker.update(numpy_detections)
            results = result_tracker

        time_after_updating_tracker = time()

        time_before_drawing_predictions = time()

        for res in results:

            x1, y1, x2, y2, obj_id = [int(a) for a  in res]


            cv2.rectangle(img, (x1, y1),(x2, y2), color=(255,0,0), thickness=2)
            cv2.putText(img,str(obj_id), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), thickness=2 )

            single_pred = [idx+1,obj_id,x1, y1, x2-x1, y2-y1,1,1,1,1]
            s = pd.Series(single_pred, index=columns)
            preds = pd.concat([s.to_frame().T, preds], ignore_index=True, axis=0)

        time_after_drawing_predictions = time()
        video.write(img)  #about 30 seconds for uav0000086_00000_v
        time_after_fps = time()

        avg_drawing_time.append((time_after_drawing_predictions-time_before_drawing_predictions)*1000)
        avg_frame_time_arr.append((time_after_fps-time_before_fps)*1000)
        avg_yolo_time_arr.append(time_after_YOLO - time_before_YOLO)
        avg_predictor_time.append(time_after_updating_tracker - time_before_updating_tracker)
        #img = image_resize(img, width=width)



    str_res = ''
    str_res+=dir_path+'\n'
    str_res+= f'Resized to {img_size}\n'
    str_res+=f'image width:{width}, image height:{height}\n'
    str_res+=f'Image reading time ms: {sum(avg_reading_time)/len(avg_reading_time)}\n'
    str_res+=f'Image resizing time ms: {sum(avg_resizing_time)/len(avg_resizing_time)}\n'
    str_res+=f'Avarage Yolo time ms:{sum(avg_yolo_time_arr)/len(avg_yolo_time_arr) * 1000}\n'
    str_res+=f'Avarage predictor time ms:{sum(avg_predictor_time)/len(avg_predictor_time) * 1000}\n'
    str_res+=f'Drawing predictions time ms: {sum(avg_drawing_time)/len(avg_drawing_time)}\n'

    #print(len(preds))
    times = write_time_to_df(video_name=dir_path_sequences,
                             resized_to=img_size,
                             source_width=width,
                             source_height=height,
                             avg_reading_time=sum(avg_reading_time)/len(avg_reading_time),
                             avg_resizing_time=sum(avg_resizing_time)/len(avg_resizing_time),
                             avg_yolo_time=sum(avg_yolo_time_arr)/len(avg_yolo_time_arr) * 1000,
                             avg_predictor_time=sum(avg_predictor_time)/len(avg_predictor_time) * 1000,
                             avg_drawing_time=sum(avg_drawing_time)/len(avg_drawing_time),
                             avg_fps_time=sum(avg_frame_time_arr)/len(avg_frame_time_arr))

    return preds, str_res, times



In [ ]:
def proccess_single_video(model=None, tracker=None, dir_path_source_sequences=None, dir_path_source_annotations=None,
                          video_name=None, img_sz=None):
    """
    proccesses one video: find time and accuracy metrics on given video

    input:
    dir_path_source_sequences - '/content/drive/MyDrive/VisDrone2019-MOT-val/sequences/'
    dir_path_source_annotations - '/content/drive/MyDrive/VisDrone2019-MOT-val/annotations/'
    video_name - name of video for both annots and sequences

    return:
    time_df - time results dataframe
    metrics_df - metrics results dataframe
    """
    print(video_name,img_sz)
    preds, res_str, time_df = make_predictions(model,tracker, dir_path_source_sequences+'/'+video_name,img_sz)

    annots = video_annot_txt_to_dataframe(f'{dir_path_source_annotations}/{video_name}.txt')

    MOT, metrics_df = motMetricsEnhancedCalculator(annots.to_numpy(), preds.to_numpy() )

    return time_df, metrics_df

In [ ]:
def proccess_all_videos():
    time_columns=['video_name','resized_to', 'source_width','source_height','avg_reading_time','avg_resizing_time',
              'avg_yolo_time','avg_predictor_time','avg_drawing_time']
    times_dataframe = pd.DataFrame(columns=time_columns)


    metric_columns = ['num_frames', 'idf1', 'idp', 'idr', 'recall', 'precision', 'num_objects',  'mostly_tracked',
                    'partially_tracked', 'mostly_lost', 'num_false_positives', 'num_misses', 'num_switches',
                    'num_fragmentations', 'mota', 'motp']
    metrics_dataframe = pd.DataFrame(columns=metric_columns)

    with open('results.txt', 'w') as f:
        for video in  os.listdir(dir_path_source_sequences):
            for s in [640]:#[640,928]

                print(video)
                print(s)

                times_df, metrics_df = proccess_single_video(model, tracker,dir_path_source_sequences,
                                                             dir_path_source_annotations,video, s)

                times_dataframe = pd.concat([times_df, times_dataframe], ignore_index=True, axis=0)
                metrics_dataframe = pd.concat([metrics_dataframe, metrics_df], ignore_index=True, axis=0)

    return times_dataframe, metrics_dataframe


In [ ]:

#!pip install ultralytics
#https://github.com/levan92/deep_sort_realtime/blob/master/deep_sort_realtime/embedder/embedder_pytorch.py
#!pip install deep-sort-realtime

In [ ]:
from sort import Sort    # crashes kernel
#from deepsort.tracker import DeepSortTracker
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
# Load a pretrained model
# Weights from https://huggingface.co/ENOT-AutoDL/yolov8s_visdrone/tree/main/enot_neural_architecture_selection_x3/weights


model = YOLO("YOLOv8s(x3).pt")

model.to('cuda');
#tracker = DeepSort(max_age=20,
#                   nms_max_overlap=0.7)
tracker = Sort()

In [ ]:
time_df, metrics_df = proccess_all_videos()

In [ ]:
time_df, metrics_df = proccess_single_video(model=model,
                                            tracker=tracker,
                                            dir_path_source_sequences=dir_path_source_sequences,
                                            dir_path_source_annotations=dir_path_source_annotations,
                                            video_name=video_name,
                                            img_sz=640)

In [ ]:
time_df['clear_fps'] = time_df['avg_reading_time'] + time_df['avg_yolo_time'] + time_df['avg_predictor_time']

In [ ]:
result = pd.concat((time_df, metrics_df), axis=1, ignore_index=False)
result.to_csv('YOLOv8_s(x3)+DeepSORT.csv', sep=';', index = False)

In [ ]:
metrics_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.heatmap(time_df.drop(['video_name'], axis=1).astype(np.float64).corr(), annot=True, annot_kws={"size": 5})
plt.show()

In [ ]:
result.sort_values('clear_fps')\
 [['resized_to','source_width',	'source_height','avg_reading_time','avg_yolo_time','avg_predictor_time','clear_fps']]

In [ ]:
f = open(dir_path_source_annotations+'\\'+video_name+'.txt')
print(len(f.readlines()))

In [ ]:
def create_videos():

    image_folder = dir_path_source_sequences
    video_folder = '\\videos'
    video_names = [img for img in os.listdir(image_folder)]
    #width,height = 10,10

    for name in video_names:
        #print(next(os.walk(image_folder+'\\'+name))[2][0])
        frame = cv2.imread(os.path.join(image_folder,name,next(os.walk(image_folder+'\\'+name))[2][0]))

        height, width, layers = frame.shape
        print(video_folder+'\\'+name)
        video = cv2.VideoWriter("D:\\Drone_object_tracking\\"+video_folder+'\\'+name+'.avi', 0, 24, (width,height))

        for img in os.listdir(image_folder+'\\'+name):
            video.write(cv2.imread(os.path.join(image_folder,name,img)))

        #print(name)

    cv2.destroyAllWindows()
    video.release()

In [ ]:
create_videos()

In [ ]:
!pip uninstall torchreid


In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import torchreid

In [8]:
from torchreid.data.datasets.video.VisDrone2019MOT import VisDrone2019MOT

torchreid.data.register_video_dataset('VisDrone2019MOT', VisDrone2019MOT)

ValueError: The given name already exists, please choose another name excluding ['mars', 'ilidsvid', 'prid2011', 'dukemtmcvidreid', 'VisDrone2019MOT']

In [12]:
datamanager = torchreid.data.VideoDataManager(
    root='D:\\Drone_object_tracking\\reid-data\\VisDrone2019-MOT-train\\uav0000013_01073_v',
    sources='VisDrone2019MOT',
    batch_size_train = 3,    #how many tracklets in a batch
    batch_size_test =3,    #how many tracklets in a batch
    num_cams = 1,
    seq_len = 15,    # how many frmaes in a tracklet
    num_instances = 4,    #how many tracklets per id in a batch
    num_datasets = 2
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded VisDrone2019MOT
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |    32 |          32 |         1
  query    |    26 |          27 |         1
  gallery  |    85 |         138 |         1
  -------------------------------------------
=> Loading test (target) dataset
=> Loaded VisDrone2019MOT
  -------------------------------------------
  subset   | # ids | # tracklets | # cameras
  -------------------------------------------
  train    |    32 |          32 |         1
  query    |    50 |          51 |         1
  gallery  

In [13]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='triplet'
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)
engine = torchreid.engine.VideoTripletEngine(
    datamanager, model, optimizer, margin=0.3,
    weight_t=1, weight_x=0, scheduler=scheduler,
    pooling_method='avg'
)
engine.run(
    max_epoch=10,
    save_dir='log/resnet50-triplet-mars',
    print_freq=10  #if (self.batch_idx + 1) % print_freq == 0:, where for self.batch_idx, data in enumerate(self.train_loader):
)
#loss_t - triplet loss
#loss_x - cross entropy loss

=> Start training
epoch: [1/10][10/10]	time 1.546 (5.374)	data 0.000 (3.883)	eta 0:08:03	loss_t 1.0016 (2.5101)	lr 0.000300
epoch: [2/10][10/10]	time 1.564 (3.098)	data 0.001 (1.636)	eta 0:04:07	loss_t 0.9001 (1.8151)	lr 0.000300
epoch: [3/10][10/10]	time 1.532 (3.279)	data 0.000 (1.775)	eta 0:03:49	loss_t 0.0841 (0.4746)	lr 0.000300
epoch: [4/10][10/10]	time 1.532 (2.987)	data 0.000 (1.525)	eta 0:02:59	loss_t 0.0000 (0.1423)	lr 0.000300
epoch: [5/10][10/10]	time 1.529 (3.153)	data 0.000 (1.660)	eta 0:02:37	loss_t 1.0189 (0.3517)	lr 0.000300
epoch: [6/10][10/10]	time 1.651 (3.248)	data 0.000 (1.748)	eta 0:02:09	loss_t 0.2032 (0.0866)	lr 0.000300
epoch: [7/10][10/10]	time 1.690 (3.272)	data 0.000 (1.764)	eta 0:01:38	loss_t 0.0000 (0.0656)	lr 0.000300
epoch: [8/10][10/10]	time 1.547 (3.246)	data 0.000 (1.787)	eta 0:01:04	loss_t 0.0013 (0.4456)	lr 0.000300
epoch: [9/10][10/10]	time 1.583 (3.113)	data 0.000 (1.640)	eta 0:00:31	loss_t 0.0013 (0.2620)	lr 0.000300
epoch: [10/10][10/10]	time 1

AssertionError: Error: all query identities do not appear in gallery

In [ ]:
video_train_root_dir = 'D:\\Drone_object_tracking\\VisDrone2019-MOT-train\\'
video_val_root_dir = 'D:\\Drone_object_tracking\\VisDrone2019-MOT-val\\'

sequences_dir = 'sequences\\'
annot_dir = 'annotations\\'

save_root_train_dir = 'D:\\Drone_object_tracking\\reid-data\\VisDrone2019-MOT-train\\'
save_root_val_dir = 'D:\\Drone_object_tracking\\reid-data\\VisDrone2019-MOT-val\\'


In [ ]:
import os.path as osp
import os

In [ ]:
from utils import video_annot_txt_to_dataframe

In [ ]:
import pandas as pd

In [ ]:

SAVE_DFS = dfs

In [ ]:
#

In [ ]:
convert_VisDrone_to_ReID()

In [ ]:
import cv2

In [ ]:

def visdrone_to_df(video_root_dir, video_annot_dir, video_seq_dir):
    """
    converts visdrone from txt to dataframes. Each video - one dataframe
    """
    dfs = []
    dirs = os.listdir(osp.join(video_root_dir,video_seq_dir))
    
    for name in dirs:
        dfs.append(video_annot_txt_to_dataframe(osp.join(video_root_dir,video_annot_dir,name+'.txt')))
        print(f'{name} converted to df')
    return dfs
#dfs.append(video_annot_txt_to_dataframe(osp.join(video_train_root_dir,annot_train_dir,name+'.txt')))
#dirs = os.listdir(osp.join(video_root_dir,sequences_train_dir))

In [ ]:
def write_from_df(dfs,video_root_dir,video_annot_dir, video_seq_dir, save_dir ):
    """
    write from dataframe to file system. Data will have next form:
    -root
    --video1
    ---id1
    ----img1
    ----img2
    ----....
    ---id2
    ----img1
    ----img2
    --video2
    ---id1
    ----img1
    ----img2
    ---id2
    ----img1
    ----img2
    ----....
    """
    dirs = os.listdir(osp.join(video_root_dir,video_seq_dir))

    for (name,df) in zip(dirs, dfs):#для 1го видео

            frame_pathes = [osp.join(video_root_dir, video_seq_dir, name, f_n) \
                            for f_n in os.listdir(osp.join(video_root_dir, video_seq_dir,name))]

            #print(frame_pathes)
            for frame_id in pd.unique(df['frame_index']):
                df_per_frame_id = df[df['frame_index'] == frame_id]

                id_counter = 1    #ids unique per video

                img = cv2.imread(frame_pathes[frame_id - 1])
                #print(img)
                for row_in_image in df_per_frame_id.iterrows():

                    f_i,target_id,left_top_x,left_top_y,width,height,score,category,truncation,occlusion=list(row_in_image)[1]

                    id_path = osp.join(save_dir,name,str(target_id))
                    if not os.path.exists(id_path):
                        os.makedirs(id_path)


                    cv2.imwrite(osp.join(id_path,str(frame_id)+'.jpg'), img[left_top_y:left_top_y+height,
                                                                            left_top_x:left_top_x+width])
                    
            print(f'{name} proccessed and written')
    

In [ ]:
def convert_train():
    dfs = visdrone_to_df(video_train_root_dir,annot_dir,sequences_dir )
    write_from_df(dfs,video_train_root_dir,annot_dir,sequences_dir,save_root_train_dir  )

In [ ]:
def convert_validation():
    dfs = visdrone_to_df(video_val_root_dir,annot_dir,sequences_dir )
    write_from_df(dfs,video_val_root_dir,annot_dir,sequences_dir,save_root_val_dir  )

In [ ]:
def clear_dataset(root):
    """
    delete id's with less than 10 frames
    """
    deleted_ids=0
    videos_dir = os.listdir(root)
    dic={}
    lst = []
    for video in videos_dir:
        video_dir = osp.join(root,video)
        idxs_dir = os.listdir(video_dir)
        for idx in idxs_dir:
            idx_dir = osp.join(video_dir,idx)
            #if len(os.listdir(idx_dir)) < 20:
            #    if video not in dic:
            #        dic[video]=1
            #    else:
            #        dic[video]+=1
            #    deleted_ids+=1
                #print(idx_dir)
            lst.append(len(os.listdir(idx_dir)))
                
        print(video)
        
    return lst

In [ ]:
root = 'D:\\Drone_object_tracking\\reid-data\\VisDrone2019-MOT-train'
numbers_of_frames_per_id = clear_dataset(root)

In [ ]:
pd.Series(numbers_of_frames_per_id).hist(bins=500)

In [ ]:
sum(numbers_of_frames_per_id)